In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
REPO_NAME = 'sewer-nfl'
import sys, os
cwd = str(os.getcwd())
repo_dir = cwd[:cwd.find(REPO_NAME)+len(REPO_NAME)]
sys.path.insert(0,repo_dir)

In [4]:
import pandas as pd
import nfl_data_py as nflreadr
from warehouse.utilities.elo import calculate_elo_metric

In [5]:
pbp_api_data = nflreadr.import_pbp_data(range(2018,2023))
pbp_api_data = pbp_api_data[pbp_api_data['week']<=18]

2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


In [6]:
roster_api_data = nflreadr.import_rosters(years=[2018,2022])

In [7]:
from warehouse.pipelines.pbp.epa import pre_elo_epa
from warehouse.utilities.elo import calculate_elo_metric

In [21]:
receiving_off_epa , receiving_def_epa = pre_elo_epa(
    pbp_api = pbp_api_data,
    roster_api = roster_api_data, 
    position_filter = ['RB','WR','TE'],
    play_types = ['pass'],
    player_id_col = 'receiver_player_id',
    order_cols = ['season','week'],
    off_gb_cols = ['posteam','defteam','position','player_name'], # At what level are we AGGREGATING offensive performance
    gb_cols_z = ['season','position'], # What is aggregate level for z score
    def_gb_cols = ['posteam','defteam','position'], # Matched up against what level on defense to AGGREGATE to
    perf_cols = ['epa'], # What are we assessing for ELO
    off_appearance_columns = ['position','receiver_player_id'],
    def_appearance_columns = ['position','defteam']
)
receiving_elo_df = calculate_elo_metric(
    input_off_data = receiving_off_epa,
    input_def_data = receiving_def_epa,
    order_cols = ['season','week'],
    off_gb_cols = ['posteam','defteam','position','player_name'], # At what level are we AGGREGATING offensive performance
    def_gb_cols = ['posteam','defteam','position'], # Matched up against what level on defense to AGGREGATE to
    off_lookup_values = ['position','receiver_player_id'],
    def_lookup_values = ['position','defteam'],
    off_perf_col = 'z_epa_x',
    def_perf_col = 'z_epa_y',
    elo_multiplier = 5,
    elo_power = 1.7,
    elo_base = 2000
)

In [22]:
receiving_elo_df

,season,week,posteam,defteam,position,player_name,receiver_player_id,z_epa_x,off_appearance,z_epa_y,def_appearance,off_elo,def_elo,off_elo_next,def_elo_next
0,2018,1,ARI,WAS,RB,Chase Edmonds,00-0034681,0.273718,1,-0.152891,1,2000,2000,2004.222339,1997.641524
1,2018,1,ARI,WAS,RB,David Johnson,00-0032187,-1.428781,1,-0.152891,1,2000,1997.641524,1977.937713,1995.285411
2,2018,1,ARI,WAS,TE,Ricky Seals-Jones,00-0033611,-1.773660,1,-1.301928,1,2000,2000,1972.639786,1979.916658
3,2018,1,ARI,WAS,WR,Chad Williams,00-0033933,-0.761107,1,0.144929,1,2000,2000,1988.259279,2002.235652
4,2018,1,ARI,WAS,WR,Christian Kirk,00-0034775,-0.807504,1,0.144929,1,2000,2002.235652,1987.555389,2004.469182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21151,2022,18,WAS,DAL,TE,Logan Thomas,00-0031260,-1.009000,56,-1.042561,83,1834.140451,1811.256998,1818.407889,1795.344417
21152,2022,18,WAS,DAL,WR,Cam Sims,00-0034104,-1.728021,36,0.108328,83,1953.824586,1330.568361,1914.886443,1333.009361
21153,2022,18,WAS,DAL,WR,Curtis Samuel,00-0033282,-0.384681,64,0.108328,83,2013.580027,1333.009361,2004.630956,1335.529468
21154,2022,18,WAS,DAL,WR,Jahan Dotson,00-0037741,0.885526,12,0.108328,83,2028.577144,1335.529468,2038.763007,1338.06549
